In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import timedelta
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [7]:
full_table = pd.read_csv('./Covid_data/covid_19_clean_complete.csv')

In [8]:
full_grouped = pd.read_csv('./Covid_data/full_grouped.csv')
full_grouped['Date'] = pd.to_datetime(full_grouped['Date'])

In [9]:
day_wise = pd.read_csv('./Covid_data/day_wise.csv')
day_wise['Date'] = pd.to_datetime(day_wise['Date'])
day_wise.head()

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287,8,6,2.76,3.83,72.22,9
3,2020-01-25,1434,42,39,1353,493,16,3,2.93,2.72,107.69,11
4,2020-01-26,2118,56,52,2010,684,14,13,2.64,2.46,107.69,13


In [10]:
country_wise = pd.read_csv('./Covid_data/country_wise_latest.csv')

country_wise = country_wise.replace('', np.nan).fillna(0)

In [11]:
worldometer_data = pd.read_csv('./Covid_data/worldometer_data.csv')
worldometer_data = worldometer_data.replace('', np.nan).fillna(0)
# worldometer_data.head()

In [12]:
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 


In [13]:
day_wise.head()

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287,8,6,2.76,3.83,72.22,9
3,2020-01-25,1434,42,39,1353,493,16,3,2.93,2.72,107.69,11
4,2020-01-26,2118,56,52,2010,684,14,13,2.64,2.46,107.69,13


In [14]:
# get the last row of the accumulated data (tail function)
temp = day_wise[ ['Date', 'Deaths', 'Recovered', 'Active']  ].tail(1)

In [15]:
day_wise[ ['Date', 'Deaths', 'Recovered', 'Active']  ]

,Date,Deaths,Recovered,Active
0,2020-01-22,17,28,510
1,2020-01-23,18,30,606
2,2020-01-24,26,36,879
3,2020-01-25,42,39,1353
4,2020-01-26,56,52,2010
...,...,...,...,...
183,2020-07-23,633506,8710969,6166006
184,2020-07-24,639650,8939705,6212290
185,2020-07-25,644517,9158743,6243930
186,2020-07-26,648621,9293464,6309711


In [16]:
temp

,Date,Deaths,Recovered,Active
187,2020-07-27,654036,9468087,6358362


In [17]:
temp = temp.melt(id_vars = 'Date', value_vars=['Active', 'Deaths', 'Recovered'])

In [18]:
temp

,Date,variable,value
0,2020-07-27,Active,6358362
1,2020-07-27,Deaths,654036
2,2020-07-27,Recovered,9468087


In [28]:
fig = px.treemap(temp, path=['variable'], values = 'value', height=225, color_discrete_sequence=[act, rec, dth])

fig.data[0].textinfo = 'label+text+value'
fig.show()

# Cases Over Time

In [5]:
def plot_daywise(col, hue):
    fig = px.bar(day_wise, x='Date', y = col, width = 700, color_discrete_sequence=[hue])
    fig.update_layout(title=col, xaxis_title='', yaxis_title='')
    fig.show()

In [116]:
def plot_daywise_line(col, hue):
    fig = px.line(day_wise, x = 'Date', y = col, width = 700, color_discrete_sequence=[hue])
    fig.update_layout(title = col, xaxis_title='', yaxis_title='')
    fig.show()

In [120]:
temp = full_grouped.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()

temp = temp.melt(id_vars = 'Date', value_vars = ['Recovered', 'Deaths', 'Active'], var_name = 'Case',
                value_name = 'Count')

temp.head()

fig = px.area(temp, x = 'Date', y = 'Count', color = 'Case', height = 600, width = 700, title = 'Case over time',
             color_discrete_sequence = [rec, dth, act])

fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

/Users/siro/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [122]:
plot_daywise('Confirmed', '#333333')

In [123]:
plot_daywise('Active', '#333333')

# plot_daywise('Deaths', dth)

In [126]:
plot_daywise('New cases', '#333333')

In [130]:
def plot_hbar(df, col, n, hover_data=[]):
    fig = px.bar(df.sort_values(col).tail(n),
                x = col, y = 'Country/Region', color = 'WHO Region',
                text = col, orientation = 'h', width = 700, hover_data = hover_data,
                color_discrete_sequence = px.colors.qualitative.Dark2)
    
    fig.update_layout(title = col, xaxis_title = '', yaxis_title='', yaxis_categoryorder= 'total ascending',
                     uniformtext_minsize = 8, uniformtext_mode = 'hide')
    fig.show()
    


In [131]:
plot_hbar(country_wise, 'Confirmed', 15)